## Installer le modèle du projet personnel: logging-of-regression-model

In [3]:
from IPython.display import clear_output
from IPython.display import Markdown as md

!pip install -e git+https://github.com/uqam-lomagnin/logging-of-regression-model-felixzhaofelix.git@main#egg=insurance_charges_model

Obtaining insurance_charges_model from git+https://github.com/uqam-lomagnin/logging-of-regression-model-felixzhaofelix.git@main#egg=insurance_charges_model
What to do?  (s)witch, (i)gnore, (w)ipe, (b)ackup ^C
ERROR: Operation cancelled by user


Importons le modèle :

In [11]:
from insurance_charges_model.prediction.model import InsuranceChargesModel

Instancions le modèle :

In [12]:
model = InsuranceChargesModel()

clear_output()

Faisons un exemple de prédiction avec l'objet InsuranceChargesModelInput :

In [13]:
from insurance_charges_model.prediction.schemas import InsuranceChargesModelInput, \
    SexEnum, RegionEnum

model_input = InsuranceChargesModelInput(
    age=42, 
    sex=SexEnum.female,
    bmi=24.0,
    children=2,
    smoker=False,
    region=RegionEnum.northwest)

Avec cet objet, nous pouvons faire une prédiction :

In [14]:
prediction = model.predict(model_input)

prediction

InsuranceChargesModelOutput(charges=8219.96)

Le modèle prédit que les frais seront de 8219,96 $. Résultat du nouveau modèle 2023 réentraîné par Groupe 1

Et on peut voir le schéma d'entrée du modèle en invoquant la méthode schema() :

In [15]:
model.input_schema.schema()

{'title': 'InsuranceChargesModelInput',
 'description': "Schema for input of the model's predict method.",
 'type': 'object',
 'properties': {'age': {'title': 'Age',
   'description': 'Age of primary beneficiary in years.',
   'minimum': 18,
   'maximum': 65,
   'type': 'integer'},
  'sex': {'title': 'Sex',
   'description': 'Gender of beneficiary.',
   'allOf': [{'$ref': '#/definitions/SexEnum'}]},
  'bmi': {'title': 'Body Mass Index',
   'description': 'Body mass index of beneficiary.',
   'minimum': 15.0,
   'maximum': 50.0,
   'type': 'number'},
  'children': {'title': 'Children',
   'description': 'Number of children covered by health insurance.',
   'minimum': 0,
   'maximum': 5,
   'type': 'integer'},
  'smoker': {'title': 'Smoker',
   'description': 'Whether beneficiary is a smoker.',
   'type': 'boolean'},
  'region': {'title': 'Region',
   'description': 'Region where beneficiary lives.',
   'allOf': [{'$ref': '#/definitions/RegionEnum'}]}},
 'definitions': {'SexEnum': {'titl

Et on aura besoin de ce schéma pour faire des fausses prédictions pour charger le modèle pour conduire le test de charge : 

## Profilage du Modèle

Afin d'avoir une idée du temps nécessaire à notre modèle pour effectuer une prédiction, nous allons le profiler en réalisant des prédictions avec des données aléatoires. Pour ce faire, nous utiliserons le package Faker : 

In [9]:
!pip install Faker


Créons une nouvelle fonction generate_record() qui va générer un enregistrement aléatoire qui respecte le schéma d'entrée du modèle :

faker génère des données aléatoires pour simuler les vraies requêtes des clients, et nous retourne un objet de type InsuranceChargesModelInput pour aliment le modèle.

In [17]:
from faker import Faker

faker = Faker()

def generate_record() -> InsuranceChargesModelInput:
    record = {
        "age": faker.random_int(min=18, max=65),
        "sex": faker.random_choices(elements=("male", "female"), length=1)[0],
        "bmi": faker.random_int(min=15000, max=50000)/1000.0,
        "children": faker.random_int(min=0, max=5),
        "smoker": faker.boolean(),
        "region": faker.random_choices(elements=("southwest", "southeast", "northwest", "northeast"), length=1)[0]
    }
    return InsuranceChargesModelInput(**record)

myInput = generate_record()   

Voici le résultat de l'exécution de la fonction :
InsuranceChargesModelInput(
age=32, 
sex=<SexEnum.male: 'male'>, 
bmi=15.444, 
children=4, 
smoker=True, 
region=<RegionEnum.northeast: 'northeast'>)

In [19]:
myPrediction = model.predict(myInput)

myPrediction

InsuranceChargesModelOutput(charges=39330.45)

Maintenant nous savons que ça marche, donc nous allons en faire mille copies aléatoires et les stocker dans une liste :

In [11]:
samples = []

for _ in range(1000):
    samples.append(generate_record())

Avec le module timeit de la bibliothèque standard, nous pouvons mesurer le temps nécessaire pour appeler la méthode de prédiction du modèle avec un échantillon aléatoire.

In [12]:
import timeit

total_seconds = timeit.timeit("[model.predict(sample) for sample in samples]", 
                              number=1, globals=globals())

seconds_per_sample = total_seconds / len(samples)
milliseconds_per_sample = seconds_per_sample * 1000.0

In [14]:
md("Le modèle a pris {} secondes pour exécuter 1000 prédictions, donc il a pris {} secondes "
   "pour faire une seule prédiction. Le modèle prend {} millisecondes pour faire une prédiction."
   .format(round(total_seconds, 2),
           round(seconds_per_sample, 3),
           round(milliseconds_per_sample,2)))

Le modèle a pris 9.01 secondes pour exécuter 1000 prédictions, donc il a pris 0.009 secondes pour faire une seule prédiction. Le modèle prend 9.01 millisecondes pour faire une prédiction.

Nous pouvons maintenant établir un SLO (Service Level Objective) pour le modèle. Disons que c'est acceptable que le modèle fasse une prédiction en moins de 100 ms. Nous pouvons formaliser cette exigence comme ceci avec assert :

In [15]:
assert milliseconds_per_sample < 100, "Model does not meet the latency SLO."

Rien n'est imprimé, donc notre modèle est conforme au critère d'exigence.

Passons à la prochaine étape pour tester le service avec plusieurs utilisateurs simultanés.

In [16]:
!pip install rest_model_service


Faisons-le fonctionner :

```bash
export PYTHONPATH=./
export REST_CONFIG=./configuration/rest_configuration.yaml
uvicorn rest_model_service.main:app --reload
```

## Créer un service de test Locust

Utilisons locust pour faire des requêtes au service de modèle. Locust est un outil de test de charge qui permet de simuler des utilisateurs simultanés qui font des requêtes à un service. Nous allons utiliser locust pour simuler des utilisateurs qui font des requêtes au service de modèle.

```bash

In [18]:
!pip install locust

clear_output()

Incorporons la méthdoe de Faker dans la classe de HttpUser de Locust  pour simuler des utilisateurs qui font des requêtes au service de modèle.




```python
from locust import HttpUser, constant_throughput, task
from faker import Faker


class ModelServiceUser(HttpUser):
    wait_time = constant_throughput(1)

    @task
    def post_prediction(self):
        faker = Faker()
        
        record = {
            "age": faker.random_int(min=18, max=65),
            "sex": faker.random_choices(elements=("male", "female"), length=1)[0],
            "bmi": faker.random_int(min=15000, max=50000) / 1000.0,
            "children": faker.random_int(min=0, max=5),
            "smoker": faker.boolean(),
            "region": faker.random_choices(
                elements=("southwest", "southeast", "northwest", "northeast"), length=1)[0]
        }
        self.client.post("/api/models/insurance_charges_model/prediction", json=record)
```
Pour refléter la réalité, ajoutons aussi progressivement des faux utilisateurs pour charger le service : 
À tous les intervalles de 30 secondes, nous allons ajouter 1 utilisateur supplémentaire jusqu'à ce que nous ayons 5 utilisateurs simultanés et revenir 
à 1 utilisateur à la fin du test avec le même rythme. Nous allons utiliser la classe LoadTestShape de locust pour définir cette forme de charge.

```python
from locust import LoadTestShape


class StagesShape(LoadTestShape):
    """Simple load test shape class."""

    stages = [
        {"duration": 30, "users": 1, "spawn_rate": 1},
        {"duration": 60, "users": 2, "spawn_rate": 1},
        {"duration": 90, "users": 3, "spawn_rate": 1},
        {"duration": 120, "users": 4, "spawn_rate": 1},
        {"duration": 150, "users": 5, "spawn_rate": 1},
        {"duration": 180, "users": 4, "spawn_rate": 1},
        {"duration": 210, "users": 3, "spawn_rate": 1},
        {"duration": 240, "users": 2, "spawn_rate": 1},
        {"duration": 270, "users": 1, "spawn_rate": 1}
    ]

    def tick(self):
        run_time = self.get_run_time()

        for stage in self.stages:
            if run_time < stage["duration"]:
                tick_data = (stage["users"], stage["spawn_rate"])
                return tick_data
        # returning None to stop the load test
        return None
```

Ajoutons 

```python
@events.test_stop.add_listener
def on_test_stop(environment, **kwargs):
    process_exit_code = 0

    max_requests_per_second = max(
        [requests_per_second for requests_per_second in environment.stats.total.num_reqs_per_sec.values()])

    if environment.stats.total.fail_ratio > 0.0:
        logger.error("Test failed because there was one or more errors.")
        process_exit_code = 1

    if environment.stats.total.get_response_time_percentile(0.99) > 100:
        logger.error("Test failed because the response time at the 99th percentile was above 100 ms. The 99th "
                     "percentile latency is '{}'.".format(environment.stats.total.get_response_time_percentile(0.99)))
        process_exit_code = 1

    if max_requests_per_second < 5:
        logger.error(
            "Test failed because the max requests per second never reached 5. The max requests per second "
            "is: '{}'.".format(max_requests_per_second))
        process_exit_code = 1

    environment.process_exit_code = process_exit_code
```


Nous ajoutons aussi la vérification des SLO suivants :

Latence : nous vérifierons que la latence au 99e centile est inférieure à 100 ms.

Taux d'erreur : nous vérifierons qu'il n'y a aucune erreur renvoyée pour toute demande.

Débit : nous vérifierons que le service peut gérer au moins 5 requêtes par seconde.

Avec une fonction listener qui reçoit des événements du package locust, nous pouvons vérifier les SLOs à la fin du test de charge. Si l'un des SLOs n'est pas respecté, nous définissons le code de sortie du processus sur 1, ce qui signale une défaillance.

```python
@events.test_stop.add_listener
def on_test_stop(environment, **kwargs):
    process_exit_code = 0

    max_requests_per_second = max(
        [requests_per_second for requests_per_second in environment.stats.total.num_reqs_per_sec.values()])

    if environment.stats.total.fail_ratio > 0.0:
        logger.error("Test failed because there was one or more errors.")
        process_exit_code = 1

    if environment.stats.total.get_response_time_percentile(0.99) > 100:
        logger.error("Test failed because the response time at the 99th percentile was above 100 ms. The 99th "
                     "percentile latency is '{}'.".format(environment.stats.total.get_response_time_percentile(0.99)))
        process_exit_code = 1

    if max_requests_per_second < 5:
        logger.error(
            "Test failed because the max requests per second never reached 5. The max requests per second "
            "is: '{}'.".format(max_requests_per_second))
        process_exit_code = 1

    environment.process_exit_code = process_exit_code
```


Lançons le service de test Locust avec le fichier python :

```bash
pwd
locust -f tests/load_test.py
```

Le Web App de Locust est accessible ici : http://127.0.0.1:8089.
Nous allons lancer le test manuellement : 



![External image](https://github.com/uqam-lomagnin/specifications-de-l-evolution-de-l-application-ml-ia_mgl7320_g1/blob/felix_load_tests/images/locust_making_requests_to_service.png)

![External image](https://github.com/uqam-lomagnin/specifications-de-l-evolution-de-l-application-ml-ia_mgl7320_g1/blob/felix_load_tests/images/service_receiving_requests_from_locust.png)

![External image](https://github.com/uqam-lomagnin/specifications-de-l-evolution-de-l-application-ml-ia_mgl7320_g1/blob/felix_load_tests/images/locust_graphs_multiple_users.png)

![External image](https://github.com/uqam-lomagnin/specifications-de-l-evolution-de-l-application-ml-ia_mgl7320_g1/blob/felix_load_tests/images/755_requests_made.png)

![External image](https://github.com/uqam-lomagnin/specifications-de-l-evolution-de-l-application-ml-ia_mgl7320_g1/blob/felix_load_tests/images/docker_image_receiving_requests.png)

In [40]:

%pwdpwd

'/Users/felixzhao/Desktop/my-logging/logging-for-ml-models'

## Faisons un test de charge avec Locust mais sans UI

La même commande de lancer le test mais sans UI pour automatiser le processus : 

```bash
locust -f tests/load_test.py --host=http://127.0.0.1:8000 --headless --loglevel ERROR --csv=./load_test_report/load_test --html ./load_test_report/load_test_report.html
```


Une fois réussi, nous pouvons voir le rapport de test de charge dans le dossier load_test_report :
Et nous pouvons faire une autre image Docker avec Dockerfile-locust pour lancer le test de charge automatiquement :

In [24]:
%cd Desktop/my-logging/logging-for-ml-models

/Users/felixzhao/Desktop/my-logging/logging-for-ml-models


/Users/felixzhao/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [41]:
!docker build -t my_locust4 -f Dockerfile-locust .

[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (2/3)                                    docker:desktop-linux
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 299B                                          0.0s
 => [internal] load build definition from Dockerfile-locust                0.0s
 => => transferring dockerfile: 796B                                       0.0s
 => [internal] load metadata for docker.io/tiangolo/uvicorn-gunicorn-fast  0.1s
[+] Building 0.3s (2/3)                                    docker:desktop-linux
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 299B                                          0.0s
 => [internal] load build definition fro

In [42]:
!docker image ls | grep my_locust4

my_locust4                                              latest            7b0c45ebd580   45 minutes ago   1.24GB


In [43]:
!docker run -p 8089:8089 my_locust4

Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated                                                                         0     0(0.00%) |      0       0       0      0 |    0.00        0.00

Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
POST     /api/models/insurance_ch

## Références

https://github.com/schmidtbri/load-tests-for-ml-models